<a href="https://colab.research.google.com/github/Hoigr/Stepik-Selenium_cours/blob/main/magnit_recom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

соревнования https://www.kaggle.com/competitions/recsys-in-practice/overview

базлайн https://colab.research.google.com/drive/1NGkBgCN7h2hMgNYtdOyirDxoZUE-Sm7g?usp=sharing

дата https://goldberg.berkeley.edu/jester-data/

In [ ]:
!pip install catboost
!pip install "scikit-surprise==1.1.3"
!pip install optuna
#!pip install joblib
#!pip install -U lightautoml

In [ ]:
import pandas as pd

from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split
from math import sqrt

import scipy.sparse as sparse
import pickle
import joblib

#import optuna #https://habr.com/ru/articles/704432/

import catboost as cb
from catboost import CatBoostRegressor

from surprise import Dataset, AlgoBase, Reader, KNNWithMeans, accuracy, SVDpp, CoClustering
from surprise.model_selection import GridSearchCV
from surprise.model_selection import KFold
from surprise.model_selection import train_test_split as tts

#from lightautoml.automl.presets.tabular_presets import TabularAutoML
#from lightautoml.tasks import Task

import tensorflow as tf
from keras.metrics import RootMeanSquaredError as RMSE
from tensorflow import keras
from keras.layers import Dense, BatchNormalization, LSTM, Dropout, Input
from keras.models import load_model
from tensorflow.python.keras import backend as K
from keras.wrappers.scikit_learn import KerasRegressor

#import torch


In [ ]:
class Data:
    def __init__(self, train:str, test:str, submit:str, full:str, gpu:bool=False):
        self.url = {'train':train, 'test':test, 'submit':submit, 'full':full}
        self.seed = 42
        self.train = pd.read_csv(train)
        self.test = pd.read_csv(test)
        self.full = pd.read_csv(full)
        self.submit = pd.read_csv(submit)
        self.uid_items = None
        self.jid_items = None
        self.cslr = None
        self.gpu = gpu
        self.implict_bpr = None
        self.implict_als = None
        self.implict_lmf = None
        self.gsKNN = None
        self.gsSVD = None
        self.gsCC = None
        self.algo = {'KNN':None, 'SVD':None, 'CC':None}
        self.automl = None
        self.learner = None
        self.catb = None
        self.kerasm = None
        self.ans = None
        
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(self.full[['UID', 'JID']],
                                                                                self.full['Rating'],
                                                                                test_size=0.2,
                                                                                random_state=self.seed)

    def metric(self):
        for i in self.train.columns:
            if i not in ('UID', 'JID', 'Rating'):
                print(f'RMSE {i}: {sqrt(mse(self.train["Rating"], self.train[i]))}')

    def load_cat(self, cat:str):
        mode = 'GPU' if self.gpu else 'CPU'
        self.catb = CatBoostRegressor(iterations=1000,
                                    learning_rate=0.1,
                                    loss_function='RMSE',
                                    task_type=mode,
                                    random_seed=self.seed)
        self.catb.load_model(cat)

    def load_gscv(self, knn:str, svd:str, cc:str):
        with open(knn, 'rb') as file:
            self.algo['KNN'] = pickle.load(file)
        with open(svd, 'rb') as file:
            self.algo['SVD'] = pickle.load(file)
        with open(cc, 'rb') as file:
            self.algo['CC'] = pickle.load(file)

    def load_lama(self, lama:str):
        self.automl = joblib.load(lama)

    def load_keras(self, ker:str):
        self.kerasm = load_model(ker)

    def load_ans(self, ans:str):
        self.ans = joblib.load(ans)

    def load_data(self, train, test, submit):
        self.train = pd.read_csv(train)
        self.test = pd.read_csv(test)
        self.submit = pd.read_csv(submit)

    def save_train_test(self):
        temp = ''
        for i in self.train.columns:
            if i not in ('Rating', 'UID', 'JID'):
                temp += i[0].upper()
        self.train.to_csv(r'/content/drive/MyDrive/DATA/MAGNIT/TT/train_predict'+temp+r'.csv')
        self.test.to_csv(r'/content/drive/MyDrive/DATA/MAGNIT/TT/test_predict'+temp+r'.csv')

    def save_submit(self):
        temp = ''
        for i in self.train.columns:
            if i not in ('Rating', 'UID', 'JID'):
                temp += i[0].upper()
        self.submit.to_csv(r'/content/drive/MyDrive/DATA/MAGNIT/out'+temp+r'.csv', index=False)

In [ ]:
class Model(Data):

    def cat(self, save=False):
        categorical_column = ['UID', 'JID']
        mode = 'GPU' if self.gpu else 'CPU'
        self.catb = CatBoostRegressor(iterations=1000, task_type=mode, verbose=True)

        self.catb.fit(self.x_train,
                    self.y_train,
                    eval_set=(self.x_test, self.y_test),
                    cat_features=categorical_column
            )
        
        self.train['cat'] = self.catb.predict(self.train[['UID', 'JID']])
        self.test['cat'] = self.catb.predict(self.test[['UID', 'JID']])
        print(f'CATBOOST RMSE : {sqrt(mse(self.train["Rating"], self.train["cat"]))}')

        if save == True:
            self.catb.save_model(r'/content/drive/MyDrive/DATA/MAGNIT/model/cat.cbm',
                                format="cbm",
                                export_parameters=None,
                                pool=None)                             

    def lama(self, save:bool=False):

        train, valid = train_test_split(self.train[['UID', 'JID', 'Rating']], 
                                        test_size=0.2,
                                        )
        self.automl = TabularAutoML(
                        task = Task(name = 'reg',
                        metric = lambda y_true, y_pred: sqrt(mse(y_true, y_pred)))
                            )
        oof_pred = self.automl.fit_predict(train, 
                                      valid_data=valid, 
                                      roles = {'target': 'Rating'}
                                      )
        
        self.train['lama'] = self.automl.predict(self.train[['UID', 'JID']]).data[:,0]
        self.test['lama'] = self.automl.predict(self.test[['UID', 'JID']]).data[:,0]
        print(f'LAMA RMSE : {sqrt(mse(self.train["Rating"], self.train["lama"]))}')
        if save == True:
            joblib.dump(self.automl, r'/content/drive/MyDrive/DATA/MAGNIT/model/lama.joblib')

    def gscv(self, save:bool=False):
        reader = Reader(rating_scale=(-10, 10))
        data = Dataset.load_from_df(self.full[['UID', 'JID', 'Rating']], reader)
        trainset, testset = tts(data, test_size=0.2, random_state=self.seed)
        
        def KNN(data, trainset, testset):
            print('Метод KNN')
            sim_options = {
                            "name": ["msd", "cosine"], # способы оценки похожести (в GridSearch)
                            "min_support": [3, 4],     # минимальное кол-во общих пользоватлей с данной шуткой
                            "user_based": [False],     # поиск "похожести" будет на основе шуток, а не пользователей
                            }

            param_grid = {"sim_options": sim_options}
            self.gsKNN = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=2,)
            self.gsKNN.fit(data)

            algo = self.gsKNN.best_estimator['rmse']
            algo.fit(trainset)
            self.algo['KNN'] = algo
            self.train['gsKNN'] = self.train[['UID', 'JID']].apply(lambda x: algo.predict(x[0], x[1], verbose=False).est,
                                                        axis = 1)
            
            self.test['gsKNN'] = self.test[['UID', 'JID']].apply(lambda x: algo.predict(x[0], x[1], verbose=False).est,
                                                        axis = 1)
            print('Метод KNN OK\n')

        def SVD(data, trainset, testset):
            print('Метод SVD++')
            param_grid = {
                'n_factors': [50],
                'n_epochs': [10],
                'lr_all': [0.005],
                'verbose': [True],
                'random_state': [self.seed]
            }

            self.gsSVD = GridSearchCV(SVDpp, param_grid, measures=["rmse", "mae"], cv=2, )
            self.gsSVD.fit(data)

            algo = self.gsSVD.best_estimator['rmse']
            algo.fit(trainset)
            self.algo['SVD'] = algo
            if save == True:
                with open(r'/content/drive/MyDrive/DATA/MAGNIT/model/gsSVD.pickle', 'wb') as file:
                    pickle.dump(algo, file)

            self.train['gsSVD'] = self.train[['UID', 'JID']].apply(lambda x: algo.predict(x[0], x[1], verbose=False).est,
                                                        axis = 1)
            
            self.test['gsSVD'] = self.test[['UID', 'JID']].apply(lambda x: algo.predict(x[0], x[1], verbose=False).est,
                                                        axis = 1)
            print('Метод SVD++ OK\n')

        def co_cluster(data, trainset, testset):
            print('Метод Co-clustering')
            param_grid = {
                'n_cltr_u': [50],
                'n_cltr_i': [50], 
                'n_epochs': [20],
                'random_state': [self.seed],
                'verbose':[True]
            }
            self.gsCC = GridSearchCV(CoClustering, param_grid, measures=["rmse", "mae"], cv=2, )
            self.gsCC.fit(data)
            algo = self.gsCC.best_estimator['rmse']
            algo.fit(trainset)
            self.algo['CC'] = algo
            self.train['gsCC'] = self.train[['UID', 'JID']].apply(lambda x: algo.predict(x[0], x[1], verbose=False).est,
                                                        axis = 1)
            
            self.test['gsCC'] = self.test[['UID', 'JID']].apply(lambda x: algo.predict(x[0], x[1], verbose=False).est,
                                                        axis = 1)
            print('Метод Co-clustering OK\n')

        KNN(data=data, trainset=trainset, testset=testset)
        SVD(data=data, trainset=trainset, testset=testset)
        co_cluster(data=data, trainset=trainset, testset=testset)
        print('KNN RMSE =', self.gsKNN.best_score["rmse"])
        print('SVD++ RMSE =', self.gsSVD.best_score["rmse"])
        print('Co-clustering RMSE =', self.gsCC.best_score["rmse"])

        if save == True:
            with open(r'/content/drive/MyDrive/DATA/MAGNIT/model/gsKNN.pickle', 'wb') as file:
                pickle.dump(self.algo['KNN'], file)
            with open(r'/content/drive/MyDrive/DATA/MAGNIT/model/gsSVD.pickle', 'wb') as file:
                pickle.dump(self.algo['SVD'], file)
            with open(r'/content/drive/MyDrive/DATA/MAGNIT/model/gsCC.pickle', 'wb') as file:
                pickle.dump(self.algo['CC'], file)

    def NN(self, save:bool=False):

        self.kerasm = keras.Sequential()
        self.kerasm.add(Dense(2, activation='linear'))
        self.kerasm.add(Dense(100, activation='tanh'))
        self.kerasm.add(Dense(100, activation='tanh'))
        self.kerasm.add(Dense(1, activation='linear'))


        self.kerasm.compile(optimizer='adam', 
                            loss = 'MSE',
                            metrics = [RMSE()])
        
        self.kerasm.fit(self.x_train[['UID', 'JID']], self.y_train, epochs=3)
        self.train['keras'] = self.kerasm.predict(self.train[['UID', 'JID']])
        self.test['keras'] = self.kerasm.predict(self.test[['UID', 'JID']])
        print(f'KERAS RMSE : {sqrt(mse(self.train["Rating"], self.train["keras"]))}')
        if save == True:
            self.kerasm.save(r'/content/drive/MyDrive/DATA/MAGNIT/model/keras.h5')

    def predict_data(self, alg:tuple):
        for i in alg:
            if i == 'cat':
                self.train['cat'] = self.catb.predict(self.train[['UID', 'JID']])
                self.test['cat'] = self.catb.predict(self.test[['UID', 'JID']])
                self.x_train['cat'] = self.catb.predict(self.x_train[['UID', 'JID']])
                self.x_test['cat'] = self.catb.predict(self.x_test[['UID', 'JID']])
            elif i == 'gscv':
                self.train['gsKNN'] = self.train[['UID', 'JID']].apply(lambda x: self.algo['KNN'].predict(x[0], x[1], verbose=False).est, axis = 1)
                self.test['gsKNN'] = self.test[['UID', 'JID']].apply(lambda x: self.algo['KNN'].predict(x[0], x[1], verbose=False).est, axis = 1)

                self.train['gsSVD'] = self.train[['UID', 'JID']].apply(lambda x: self.algo['SVD'].predict(x[0], x[1], verbose=False).est, axis = 1)
                self.test['gsSVD'] = self.test[['UID', 'JID']].apply(lambda x: self.algo['SVD'].predict(x[0], x[1], verbose=False).est, axis = 1)

                self.train['gsCC'] = self.train[['UID', 'JID']].apply(lambda x: self.algo['CC'].predict(x[0], x[1], verbose=False).est, axis = 1)
                self.test['gsCC'] = self.test[['UID', 'JID']].apply(lambda x: self.algo['CC'].predict(x[0], x[1], verbose=False).est, axis = 1)
            elif i == 'lama':

                self.train['lama'] = self.automl.predict(self.train[['UID', 'JID', 'Rating']]).data[:,0]
                self.test['lama'] = self.automl.predict(self.test[['UID', 'JID', 'Rating']]).data[:,0]
            elif i == 'keras':
                self.train['keras'] = self.kerasm.predict(self.train[['UID', 'JID']])
                self.test['keras'] = self.kerasm.predict(self.test[['UID', 'JID']])
            elif i == 'ans':
                test_col = list(self.train.columns)
                test_col.remove('UID')
                test_col.remove('JID')
                test_col.remove('InteractionID')
                train_col = list(self.train.columns)
                train_col.remove('UID')
                train_col.remove('JID')                
                train_col.remove('Rating')
                self.train['lama'] = self.ans.predict(self.train[train_col]).data[:,0]
                self.test['lama'] = self.ans.predict(self.test[test_col]).data[:,0]
            else:
                print(i, 'Некоректное имя модели')

    def cat_ans(self, save:bool=False):
        mode = 'GPU' if self.gpu else 'CPU'
        self.ansm = CatBoostRegressor(iterations=10, task_type=mode, verbose=True)
        col_train = ['gsKNN', 'gsSVD', 'gsCC']
        x_train, x_test, y_train, y_test = train_test_split(self.train[col_train],
                                                            self.train['Rating'],
                                                            test_size=0.2,
                                                            random_state=self.seed)

        self.ansm.fit(self.x_train,
                self.y_train,
                eval_set=(self.x_test, self.y_test),
            )

        self.train['an'] = self.ansm.predict(self.train[['gsKNN', 'gsSVD', 'gsCC']])
        #self.test['an'] = self.ansm.predict(self.test[col_train])
        #self.submit['Rating'] = self.test['an']
        print(f'CATBOOST ANS RMSE : {sqrt(mse(self.train["Rating"], self.train["an"]))}')
        if save == True:
            self.ansm.save_model(r'/content/drive/MyDrive/DATA/MAGNIT/model/ans.cbm',
                                format="cbm",
                                export_parameters=None,
                                pool=None)
            
    def ansib(self, save:bool=False):
        train_col = list(self.train.columns)
        train_col.remove('UID')
        train_col.remove('JID')
        test_col = list(self.test.columns)
        test_col.remove('InteractionID')
        test_col.remove('UID')
        test_col.remove('JID')
        print(train_col)
        print(test_col)

        train, valid = train_test_split(self.train, 
                                        test_size=0.2,
                                        )
        self.ans = TabularAutoML(
                        task = Task(name = 'reg',
                        metric = lambda y_true, y_pred: sqrt(mse(y_true, y_pred)))
                            )
        oof_pred = self.ans.fit_predict(train, 
                                      valid_data=valid, 
                                      roles = {'target': 'Rating'}
                                      )
        
        self.train['ans'] = self.ans.predict(self.train[train_col]).data[:,0]
        self.test['ans'] = self.ans.predict(self.test[test_col]).data[:,0]
        self.submit['Rating'] = self.test['ans']
        if save == True:
            joblib.dump(self.ans, r'/content/drive/MyDrive/DATA/MAGNIT/model/ansib.joblib')

    def fin(self):
        # categorical_column = ['UID', 'JID']
        # cat_features=categorical_column
        model = CatBoostRegressor(iterations=1000, verbose=True)
        col_train = ['gsKNN', 'gsSVD', 'gsCC', 'cat']
        col_test = ['gsKNN', 'gsSVD', 'gsCC', 'cat']
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(self.train[col_train],
                                                                                self.train['Rating'],
                                                                                test_size=0.2,
                                                                                random_state=self.seed)                    
        model.fit(self.x_train,
                    self.y_train,
                    eval_set=(self.x_test, self.y_test),
                    
            )
        self.train['fin'] = model.predict(self.train[col_train])
        self.test['fin'] = model.predict(self.test[col_test])
        self.submit['Rating'] = self.test['fin']       

In [ ]:
full_data = r'/content/drive/MyDrive/DATA/MAGNIT/jester_ratings.csv'
train = r'/content/drive/MyDrive/DATA/MAGNIT/train_joke_df.csv'
test = r'/content/drive/MyDrive/DATA/MAGNIT/test_joke_df_nofactrating.csv'
submit = r'/content/drive/MyDrive/DATA/MAGNIT/sample_submission.csv'
gpu = False
zn = Model(train=train, test=test, submit=submit, full=full_data, gpu=gpu)
%time zn.gscv(save=False) # длительность ~ 15 минут + 13 минут + 13 минут CPU
zn.save_train_test()
%time zn.cat(save=False) # длительность ~ 8 минут CPU/GPU
zn.save_train_test()
%time zn.fin()
#%time zn.lama(save=False) # длительность ~ 30 минут CPU
#zn.save_train_test()
#%time zn.NN(save=False) # длительность ~ 30 минут CPU
#zn.save_train_test()
#%time zn.cat_ans(save=False) 
zn.metric()
zn.save_train_test()
zn.save_submit()

In [ ]:
zn.test

In [ ]:
train = r'/content/drive/MyDrive/DATA/MAGNIT/TT/train_predictGGGC.csv'
test = r'/content/drive/MyDrive/DATA/MAGNIT/TT/test_predictGGGC.csv'
submit = r'/content/drive/MyDrive/DATA/MAGNIT/sample_submission.csv'
gpu = False
zn = Model(train=train, test=test, submit=submit, gpu=gpu)
zn.train = zn.train.drop(['Unnamed: 0'], axis=1)
zn.test = zn.test.drop(['Unnamed: 0'], axis=1)
zn.fin()
zn.metric()
zn.save_submit()

In [ ]:
full_data = r'/content/drive/MyDrive/DATA/MAGNIT/jester_ratings.csv'
train = r'/content/drive/MyDrive/DATA/MAGNIT/train_joke_df.csv'
test = r'/content/drive/MyDrive/DATA/MAGNIT/test_joke_df_nofactrating.csv'
submit = r'/content/drive/MyDrive/DATA/MAGNIT/sample_submission.csv'
gpu = False
zn = Model(train=train, test=test, submit=submit, gpu=gpu)
# Загрузка модели Surprise
url_knn = r'/content/drive/MyDrive/DATA/MAGNIT/model/gsKNN.pickle'
url_svd = r'/content/drive/MyDrive/DATA/MAGNIT/model/gsSVD.pickle'
url_cc = r'/content/drive/MyDrive/DATA/MAGNIT/model/gsCC.pickle'
zn.load_gscv(knn=url_knn, svd=url_svd, cc=url_cc)
url_cat = r'/content/drive/MyDrive/DATA/MAGNIT/model/cat.cbm'
zn.load_cat(cat=url_cat)
zn.predict_data(('gscv','cat'))
#%time zn.NN(save=True)
%time zn.cat_ans(save=True) 
zn.save_train_test()
zn.metric()
zn.save_submit()

In [ ]:
url_knn = r'/content/drive/MyDrive/DATA/MAGNIT/model/gsKNN.pickle'
url_svd = r'/content/drive/MyDrive/DATA/MAGNIT/model/gsSVD.pickle'
url_cc = r'/content/drive/MyDrive/DATA/MAGNIT/model/gsCC.pickle'
zn.load_gscv(knn=url_knn, svd=url_svd, cc=url_cc)
url_cat = r'/content/drive/MyDrive/DATA/MAGNIT/model/cat.cbm'
zn.load_cat(cat=url_cat)
zn.predict_data(('gscv','cat'))

%time zn.cat_ans(save=True) 
zn.save_train_test()
zn.metric()
zn.save_submit()

In [ ]:
# Загрузка модели catboost
url_cat = r'/content/drive/MyDrive/DATA/MAGNIT/model/cat.cbm'
zn.load_cat(cat=url_cat)

In [ ]:
# Загрузка модели Surprise
url_knn = r'/content/drive/MyDrive/DATA/MAGNIT/model/gsKNN.pickle'
url_svd = r'/content/drive/MyDrive/DATA/MAGNIT/model/gsSVD.pickle'
url_cc = r'/content/drive/MyDrive/DATA/MAGNIT/model/gsCC.pickle'
zn.load_gscv(knn=url_knn, svd=url_svd, cc=url_cc)

In [ ]:
# Загрузка модели LAMA
# пока не работает
# https://you.com/search?q=lama+automl+save+model&fromSearchBar=true&tbm=youchat&cid=c2_34ea8efb-e0ac-4642-ba06-9494b1a69a6a
url_lama = r'/content/drive/MyDrive/DATA/MAGNIT/model/lama.joblib'
zn.load_lama(lama=url_lama)

In [ ]:
# Загрузка модели KERAS
url_keras = r'/content/drive/MyDrive/DATA/MAGNIT/model/keras.h5'
zn.load_keras(ker=url_keras)

In [ ]:
# Загрузка модели ансамбля
# пока не работает
url_ans = r'/content/drive/MyDrive/DATA/MAGNIT/model/ans.joblib'
zn.load_ansib(ans=url_ans)